# Day 12 - Passage Pathing

https://adventofcode.com/2021/day/12

In [57]:
from pathlib import Path
from collections import defaultdict

INPUTS = Path("input.txt").read_text().strip().split("\n")


## Part 1

To start, we need a way to process the inputs into a "node map", which for our needs is simply a dict where every key is a cave and their values are lists of nodes that cave can connect to.

The inputs are arranged in random ordering, and there is no direction to each connection, so we need to run the same connections from `left->right` as `right->left`. For instance, `A-b` is a connection from `A` to `b` *and* from `b` to `A`.

To makes things a little easier down the road, we set a couple rules for the mapping:

- "end" won't be a starting node, as we'll never branch away from the end point.
- "start" can never be a destination, as we'll always begin there and never revisit it.

In [58]:
def get_node_map(inputs: list[str] = INPUTS):
    node_map = defaultdict(list)
    for line in inputs:
        left, right = line.split("-")
        if right != "start" and left != "end":
            node_map[left].append(right)
        if left != "start" and right != "end":
            node_map[right].append(left)
    return node_map


NODES = get_node_map()


Now for the algorithm, which is a relatively simple recursive function checking all branching pathways from one node to all its destinations.

- Start a number of `paths` from the current node at 0. If we end up in a dead branch, this will never increment.
- If one of our destinations is `end`, increment by 1 and continue to the next destination. There is no need to traverse recursively down this path.
- If it's a small cave, we cannot visit each more than once, so check that the cave is in a running list of `visited` caves. As we recurse, we add the current cave to this list to keep track of where we've been on the current branch. So, if the small cave has already been visited, skip it.
- With all above tests passed, call the function recursively to get the number of branches from the given destination.

In [59]:
import string
from copy import deepcopy


def node_is_small(node: str) -> bool:
    return node != 'start' and node[0] in string.ascii_lowercase


def num_paths(
    node_map: dict[str, list] = NODES,
    current: str = "start",
    visited: list = None,
) -> int:
    paths = 0
    if visited is None:
        visited = []
    new_visited = deepcopy(visited)
    new_visited.append(current)
    for node in node_map[current]:
        if node == "end":
            paths += 1
            continue
        if node_is_small(node=node) and node in new_visited:
            continue
        paths += num_paths(node_map=node_map, current=node, visited=new_visited)
    return paths


Testing is important to know that our method matches expectations, so we run it against all the examples provided on the AoC site:

In [60]:
def test_num_paths():
    map1 = [
        "start-A",
        "start-b",
        "A-c",
        "A-b",
        "b-d",
        "A-end",
        "b-end",
    ]
    node_map1 = get_node_map(inputs=map1)
    paths1 = num_paths(node_map=node_map1)
    assert paths1 == 10

    map2 = [
        "dc-end",
        "HN-start",
        "start-kj",
        "dc-start",
        "dc-HN",
        "LN-dc",
        "HN-end",
        "kj-sa",
        "kj-HN",
        "kj-dc",
    ]
    node_map2 = get_node_map(inputs=map2)
    paths2 = num_paths(node_map=node_map2)
    assert paths2 == 19

    map3 = [
        "fs-end",
        "he-DX",
        "fs-he",
        "start-DX",
        "pj-DX",
        "end-zg",
        "zg-sl",
        "zg-pj",
        "pj-he",
        "RW-he",
        "fs-DX",
        "pj-RW",
        "zg-RW",
        "start-pj",
        "he-WI",
        "zg-he",
        "pj-fs",
        "start-RW",
    ]
    node_map3 = get_node_map(inputs=map3)
    paths3 = num_paths(node_map=node_map3)
    assert paths3 == 226


test_num_paths()


With passing tests, we can now grab our solution.

In [61]:
paths = num_paths()
print(f"Number of paths in the map: {paths}")

Number of paths in the map: 3563


## Part 2

In addition to checking if a small node was previously visited, we now need to test if *any* small node was already visited twice in some branching path. If not, let that test pass and continue continuing branches as normal.

Just needs a helper function for checking if the `visited` list contains a "double small":

In [62]:
def contains_double_small_visit(visited: list[str]) -> bool:
    smalls_visited = defaultdict(int)
    for cave in visited:
        if node_is_small(cave):
            smalls_visited[cave] += 1
    double_visits = [k for k, v in smalls_visited.items() if v > 1]
    return bool(double_visits)


...and then we should be good to go. Note the only real change below is the new test using `contains_double_small_visit`. If that returns `False`, we allow it to recurse down the path to the small cave again just like any other visit. On subsequent tests down that pathway, the same test will then return `True` and skip it, as it should.

In [63]:
def num_paths2(
    node_map: dict[str, list] = NODES,
    current: str = "start",
    visited: list = None,
) -> int:
    paths = 0
    if visited is None:
        visited = []
    new_visited = deepcopy(visited)
    new_visited.append(current)
    for node in node_map[current]:
        if node == "end":
            paths += 1
            continue
        if (
            node_is_small(node=node)
            and node in new_visited
            and contains_double_small_visit(visited=new_visited)
        ):
            continue
        paths += num_paths2(node_map=node_map, current=node, visited=new_visited)
    return paths


Again some sanity checks.

In [64]:
def test_num_paths2():
    map1 = [
        "start-A",
        "start-b",
        "A-c",
        "A-b",
        "b-d",
        "A-end",
        "b-end",
    ]
    node_map1 = get_node_map(inputs=map1)
    paths1 = num_paths2(node_map=node_map1)
    assert paths1 == 36, f"{paths1=}"

    map2 = [
        "dc-end",
        "HN-start",
        "start-kj",
        "dc-start",
        "dc-HN",
        "LN-dc",
        "HN-end",
        "kj-sa",
        "kj-HN",
        "kj-dc",
    ]
    node_map2 = get_node_map(inputs=map2)
    paths2 = num_paths2(node_map=node_map2)
    assert paths2 == 103, f"{paths2=}"

    map3 = [
        "fs-end",
        "he-DX",
        "fs-he",
        "start-DX",
        "pj-DX",
        "end-zg",
        "zg-sl",
        "zg-pj",
        "pj-he",
        "RW-he",
        "fs-DX",
        "pj-RW",
        "zg-RW",
        "start-pj",
        "he-WI",
        "zg-he",
        "pj-fs",
        "start-RW",
    ]
    node_map3 = get_node_map(inputs=map3)
    paths3 = num_paths2(node_map=node_map3)
    assert paths3 == 3509, f"{paths3=}"


test_num_paths2()

And then find our solution.

In [65]:
paths2 = num_paths2()
print(f"Number of paths in the map (with double visits): {paths2}")